# 1. Backpropagation for bias vectors (1 point)

In class, we discussed a multilayer perceptron (neural network) whose layers were all "dense", i.e. the output $a^m \in \mathbb{R}^{N^m}$ of the $m$th layer is computed as 
\begin{align*}
z^m &= W^m a^{m - 1} + b^m \\
a^m &= \sigma^m(z^m)
\end{align*}
where $W^m \in \mathbb{R}^{N^m \times N^{m - 1}}$ is the weight matrix, $b^m \in \mathbb{R}^{N^m}$ is the bias vector, and $\sigma^m$ is the nonlinearity. We showed that 
$$\frac{\partial C}{\partial W^m} = \frac{\partial C}{\partial z^m} a^{m - 1 \top}$$
Show that
$$\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}$$
Hint: The derivation is almost the same as for $W$.

**Solution:**

$$
\frac{\partial C}{\partial b^m} = 
\frac{\partial C}{\partial b^m} = \sum_{k=1}^{N}\frac{\partial C}{\partial z^m_{k}} \frac{\partial z^m_{k}}{\partial b^m_{ij}}
$$

$$
z^m_k = \sum_{e=1}^{N^m} W_{k,e}^m a_e^{m - 1} + b_{k}^m
$$

If $i = k$,
$$
\frac{\partial z_i^m}{\partial b_{i,j}^m} = \frac{\partial \sum_{e=1}^{N^m} (W_{i,e}^m a_e^{m - 1} + b_{i}^m)}{\partial b_{i,j}^m} = 1^m
$$

If $i \not= k $, 
$$
\frac{\partial z_i^m}{\partial b_{i,j}^m} = 0
$$

Therefore,
$$
\frac{\partial C}{\partial b^m} = 
\frac{\partial C}{\partial z^m}
$$

# 2. MLP from scratch (3 points)

Using numpy only, implement backward pass or a sigmoid MLP. Specifically, you will need to implement this functionality in the `train` function in the `SigmoidMLP` class below. You should write numpy code to populate the two lists `weight_gradients` and `bias_gradient`, where each entry in each list corresponds to the gradient for a weight matrix or bias vector for each layer. Then, when you run the code cell at the bottom of this notebook, the trained MLP should output (approximately) 0, 1, 1, 0, having learned the [XOR function](https://en.wikipedia.org/wiki/Exclusive_or). Please us a binary cross-entropy loss, i.e.
$$C(a^L, y) = (y - 1)\log(1 - a^L) - y\log(a^L)$$

*Note 1*: All layers in your model, including the last layer, will use the sigmoid cross-entropy function. Remember that 
$$
\frac{\partial}{\partial x}\mathrm{sigmoid}(x) = \mathrm{sigmoid}(x)(1 - \mathrm{sigmoid}(x))$$

*Note 2*: As we mentioned in class,
$$
\frac{\partial C}{\partial z^L} = a^L - y
$$

In [31]:
import numpy as np

In [32]:
class Layer:
    def __init__(self, inputs, outputs):
        # Initialize weight matrix and bias vector
        # Getting the initialization right can be tricky, but for this problem
        # simply drawing from a standard normal distribution should work.
        self.weights = np.random.randn(outputs, inputs)
        self.biases = np.random.randn(outputs, 1)
    def __call__(self, X):
        # Compute \sigmoid(Wx + b)
        return 1/(1 + np.exp(-(self.weights.dot(X) + self.biases)))

In [33]:
class SigmoidMLP:

    def __init__(self, layer_widths):
        self.layers = []
        for inputs, outputs in zip(layer_widths[:-1], layer_widths[1:]):
            self.layers.append(Layer(inputs, outputs))
    
    def train(self, inputs, targets, learning_rate):
        # Forward pass - compute each layer's output and store it for later use
        layer_outputs = [inputs]
        for layer in self.layers:
            layer_outputs.append(layer(layer_outputs[-1]))
        
        # Implement backward pass to populate weight_gradients and bias_gradients
        # lists here
        weight_gradients = []
        bias_gradients = []
        # ... (your code here) ...


        # ------------------------------Method 1: Each layer is calculated separately--------------------
        # # output layer
        # weight_gradient_2 = np.dot((layer_outputs[2] - targets), layer_outputs[1].T)
        # bias_gradient_2 = layer_outputs[2] - targets
        # bias_gradient_2 = (np.mean(bias_gradient_2, 1)).reshape(self.layers[1].biases.shape)  # reshape(1, 1)

        # # hidden layer
        # weight_gradient_1 = np.dot(np.dot(self.layers[-1].weights.T , (layer_outputs[2] - targets)) * layer_outputs[1] * (1 - layer_outputs[1]) , layer_outputs[0].T)
        # bias_gradient_1 = np.dot(self.layers[-1].weights.T, (layer_outputs[2] - targets)) * layer_outputs[1] * (1 - layer_outputs[1])
        # bias_gradient_1 = (np.mean(bias_gradient_1, 1)).reshape(self.layers[0].biases.shape)  # reshape(2, 1)

        # # add the gradients to lists
        # weight_gradients.append(weight_gradient_1)
        # weight_gradients.append(weight_gradient_2)
        # bias_gradients.append(bias_gradient_1)
        # bias_gradients.append(bias_gradient_2)
        # -----------------------------------------Method 1 end-----------------------------------------------------


        # ------------------Method 2: Use loop-----------------------
        # dc_dzl = layer_outputs[-1] - targets
        # dc_dzm = [dc_dzl]

        # dc_dwl = np.dot(dc_dzl, layer_outputs[-2].T)
        # dc_dwm = [dc_dwl]

        # dc_dbl = np.mean(dc_dzl, 1).reshape(self.layers[-1].biases.shape)
        # dc_dbm = [dc_dbl]

        # for i in range(len(self.layers), -1, -1):
        #     # dc_dzm 
        #     dc_dz = np.dot(self.layers[i-1].weights.T, dc_dzm[-1]) * layer_outputs[i-1] * (1 - layer_outputs[i-1])
        #     dc_dzm.append(dc_dz)
        #     # dc_dwm
        #     dc_dw = np.dot(dc_dz, layer_outputs[i-2].T)
        #     dc_dwm.append(dc_dw)
        #     # dc_dbm
        #     dc_db = np.mean(dc_dz, 1).reshape(self.layers[i-2].biases.shape)
        #     dc_dbm.append(dc_db)
        #     # end loop
        #     if i == 2:
        #         break
        
        # # reverse the gradients
        # weight_gradients = reversed(dc_dwm)
        # bias_gradients = reversed(dc_dbm)
        # -------------------------------------------- Method 2 end ----------------------------------------
        

        # ---------------------------------------- Method 3: Optimization of method 2 ---------------------
        dc_dzl = layer_outputs[-1] - targets
        dc_dzm = dc_dzl

        for i in range(len(self.layers), 0, -1):
            dc_dwm = np.dot(dc_dzm, layer_outputs[i-1].T)
            dc_dbm = np.mean(dc_dzm, 1).reshape(self.layers[i-1].biases.shape)
            dc_dzm = np.dot(self.layers[i-1].weights.T, dc_dzm) * layer_outputs[i-1] * (1 - layer_outputs[i-1])

            weight_gradients.append(dc_dwm)
            bias_gradients.append(dc_dbm)
        
        weight_gradients.reverse()
        bias_gradients.reverse()
        # ---------------------------------------- Method 3 end -----------------------------


        # Perform gradient descent by applying updates
        for weight_gradient, bias_gradient, layer in zip(weight_gradients, bias_gradients, self.layers):
            layer.weights -= weight_gradient*learning_rate
            layer.biases -= bias_gradient*learning_rate

    def __call__(self, inputs):
        a = inputs
        for layer in self.layers:
            a = layer(a)
        return a


In [34]:
def train_mlp(n_iterations, learning_rate):
    mlp = SigmoidMLP([2, 2, 1])
    inputs = np.array([[0, 1, 0, 1], 
                       [0, 0, 1, 1]])
    targets = np.array([[0, 1, 1, 0]])
    for _ in range(n_iterations):
        mlp.train(inputs, targets, learning_rate)
    return mlp

In [35]:
# You may need to change the n_iterations and learning_rate values
# but these worked for me
mlp = train_mlp(1000, 1.)
# The following calls should result in (approximately) 0, 1, 1, 0
# If the outputs are somewhat close, your training has succeeded!
print(mlp(np.array([0, 0]).reshape(-1, 1)))
print(mlp(np.array([0, 1]).reshape(-1, 1)))
print(mlp(np.array([1, 0]).reshape(-1, 1)))
print(mlp(np.array([1, 1]).reshape(-1, 1)))

[[0.00706881]]
[[0.99715013]]
[[0.99715001]]
[[0.00320043]]
